In [2]:
import os
import shutil
import subprocess
from pathlib import Path
import re
import csv
import pydicom


incoming_dir = Path("incoming_dicoms")
processed_dir = Path("processed_dicoms")
bids_output_dir = Path("bids_output")
config_file = Path("dcm2bids.config.json")
subject_map_csv = Path("bids_output/subject_map.csv")  # CSV file to track subject info


processed_dir.mkdir(parents=True, exist_ok=True)


# === Get the next BIDS subject number ===
def get_next_subject_number(bids_dir):
    existing = [d.name for d in bids_dir.glob("sub-*")]
    numbers = [int(re.findall(r'\d+', name)[0]) for name in existing if re.match(r"sub-\d+", name)]
    return max(numbers, default=0) + 1


# === Extract age, sex, and scan date from DICOM headers ===
def extract_dicom_metadata(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".dcm"):
                try:
                    dcm = pydicom.dcmread(os.path.join(root, file), stop_before_pixels=True)
                    age = getattr(dcm, "PatientAge", "NA")
                    sex = getattr(dcm, "PatientSex", "NA")
                    date = getattr(dcm, "StudyDate", "NA")
                    return age, sex, date
                except Exception:
                    continue
    return "NA", "NA", "NA"


# === Update CSV mapping of subject info ===
def update_subject_map(csv_file, patient_folder_name, subject_label, age, sex, scan_date):
    rows = []

    if csv_file.exists():
        with open(csv_file, "r", newline="") as f:
            reader = csv.reader(f)
            rows = list(reader)

        # Skip if already logged
        for row in rows:
            if row[0] == patient_folder_name:
                return

    with open(csv_file, "a", newline="") as f:
        writer = csv.writer(f)
        if not rows:
            writer.writerow(["OriginalFolder", "BIDS_Subject", "Age", "Sex", "ScanDate"])
        writer.writerow([patient_folder_name, f"sub-{subject_label}", age, sex, scan_date])


# === Process all unprocessed folders ===
for patient_folder in incoming_dir.iterdir():
    if not patient_folder.is_dir():
        continue

    # Skip if already marked as converted
    if (patient_folder / "converted.flag").exists():
        continue

    # Determine next subject label
    sub_num = get_next_subject_number(bids_output_dir)
    sub_label = f"{sub_num:06d}"

    print(f"\nProcessing folder: {patient_folder.name} → sub-{sub_label}")

    # Extract DICOM metadata
    age, sex, scan_date = extract_dicom_metadata(patient_folder)

    # Construct dcm2bids command
    cmd = [
        "dcm2bids",
        "-d", str(patient_folder),
        "-p", sub_label,
        "-c", str(config_file),
        "-o", str(bids_output_dir),
        "--force_dcm2bids",
        "--clobber",
        "-l", "DEBUG"
    ]

    try:
        subprocess.run(cmd, check=True)

        # Mark as processed
        (patient_folder / "converted.flag").touch()

        # Update CSV
        update_subject_map(subject_map_csv, patient_folder.name, sub_label, age, sex, scan_date)

        # Move folder to processed_dir
        dest_folder = processed_dir / patient_folder.name
        if dest_folder.exists():
            print(f"Destination {dest_folder} already exists. Skipping move.")
        else:
            shutil.move(str(patient_folder), str(dest_folder))
            print(f"Moved {patient_folder.name} to processed folder.")

    except subprocess.CalledProcessError:
        print(f"Error processing {patient_folder.name}. Skipping.")



Processing folder: 865SK_B → sub-000001
INFO    | --- dcm2bids start ---
INFO    | Running the following command: /opt/miniconda3/envs/bidsenv/bin/dcm2bids -d incoming_dicoms/865SK_B -p 000001 -c dcm2bids.config.json -o bids_output --force_dcm2bids --clobber -l DEBUG
INFO    | OS version: macOS-15.6-arm64-arm-64bit
INFO    | Python version: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
INFO    | dcm2bids version: 3.2.0
INFO    | dcm2niix version: v1.0.20240202
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids.
WARNING | A newer version exists for dcm2niix: v1.0.20250506
WARNING | You should update it -> https://github.com/rordenlab/dcm2niix.
INFO    | participant: sub-000001
INFO    | config: /Users/PHARAOH/Downloads/Processing/dcm2bids.config.json
INFO    | BIDS directory: /Users/PHARAOH/Downloads/Processing/bids_output
INFO    | Auto extract entities: False
INFO    | Reorder entities: True
INFO    | Validate BIDS: False

INFO    